In [1]:
import pandas as pd 

In [2]:
expert_agent = pd.read_csv("data/data_generation_mpc_110_190_6_all.csv")

### Split the data in Features and Targets and Scale using Sklearn

In [3]:
from sklearn import preprocessing

X = expert_agent[["inflow", "height"]]
X_scaler = preprocessing.StandardScaler().fit(X)
X_scaled = X_scaler.transform(X)

y = expert_agent[["speed1_rpm", "speed4_rpm"]]
y_scaler = preprocessing.StandardScaler().fit(y)
y_scaled = y_scaler.transform(y)


### Transform arrays into Torch DataLoader

In [4]:
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import random_split
import torch

seed = torch.Generator().manual_seed(42)

torch_dataset = TensorDataset(torch.tensor(X_scaled, dtype=torch.float32), 
                              torch.tensor(y_scaled, dtype=torch.float32))

total_size = len(torch_dataset)
train_size = int(0.7 * total_size)  # 70% of data for training
valid_size = int(0.15 * total_size)  # 15% of data for validation
test_size = total_size - train_size - valid_size  # Remaining 15% for testing

train_dataset, valid_dataset, test_dataset = random_split(torch_dataset, [train_size, valid_size, test_size], generator=seed)

# Create Data Loaders for each set
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

### Define the NN Skeleton in Pytorch

In [5]:
import pytorch_lightning as pl


import torch
import torch.nn as nn
import torch.optim as optim

# Define the dataset (assuming X_scaled, y_scaled are numpy arrays)
train_dataset = TensorDataset(torch.tensor(X_scaled, dtype=torch.float32), 
                              torch.tensor(y_scaled, dtype=torch.float32))

# Define the Neural Network using PyTorch Lightning ==> subclassing
class LitNeuralNet(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 50)
        self.fc2 = nn.Linear(50, 10)
        self.fc3 = nn.Linear(10, 2)
        self.train_loss = []
        self.val_loss = []

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        train_loss = nn.functional.mse_loss(y_hat, y)
        self.train_loss.append(train_loss.cpu().item())
        self.log('train_loss', train_loss, on_epoch=True, prog_bar=True)
        return train_loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        val_loss = nn.functional.mse_loss(y_hat, y)
        self.val_loss.append(val_loss.cpu().item())
        self.log('val_loss', val_loss, on_epoch=True, prog_bar=True)
        return val_loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        test_loss = nn.functional.mse_loss(y_hat, y)
        self.log('test_loss', test_loss, on_epoch=True)
        return test_loss

    def configure_optimizers(self):
        # Add L2 regularization with weight_decay
        optimizer = optim.Adam(self.parameters(), lr=1e-3, weight_decay=1e-5)
        return optimizer

### Initialize the NN class

In [6]:
model = LitNeuralNet()

### Create Model Early Stopping to avoid Overfitting

In [7]:
from pytorch_lightning.callbacks import EarlyStopping
# Early stopping callback
early_stop_callback = EarlyStopping(monitor='val_loss', patience=3, verbose=True, mode='min')

In [8]:
from pytorch_lightning.callbacks import ModelCheckpoint

# Define a ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',     # Monitor validation loss for checkpointing
    dirpath='checkpoints/', # Directory where checkpoints will be saved
    filename='model-{epoch:02d}-{val_loss:.2f}', # Checkpoint file name
    save_top_k=3,           # Save top 3 models based on val_loss
    mode='min',             # 'min' mode saves the model when val_loss is minimized
)

### Optional: Use Weights and Bias website to visualize the stats of the training phase.

In [9]:
from pytorch_lightning.loggers import WandbLogger
wandb_logger = WandbLogger(log_model="all")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alqua. Use `wandb login --relogin` to force relogin


### Train the Neural Network

In [10]:

# Trainer
trainer = pl.Trainer(callbacks=[early_stop_callback],
                     logger=wandb_logger, 
                     accelerator= 'cpu',
                     max_epochs=20)

# Train the model
trainer.fit(model, train_dataloaders= train_loader, val_dataloaders= valid_loader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/alqua/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 150   
1 | fc2  | Linear | 510   
2 | fc3  | Linear | 22    
--------------------------------
682       Trainable params
0         Non-trainable params
682       Total params
0.003     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/alqua/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/alqua/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.001


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


In [11]:
trainer.test(dataloaders=test_loader)

/home/alqua/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:148: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
Restoring states from the checkpoint path at ./lightning_logs/258xosn4/checkpoints/epoch=19-step=28900.ckpt
Loaded model weights from the checkpoint at ./lightning_logs/258xosn4/checkpoints/epoch=19-step=28900.ckpt
/home/alqua/anaconda3/envs/pytorch_env/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machin

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.testing metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss          0.001409763004630804
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.001409763004630804}]

### Save the Scaler for Future use

In [15]:
import joblib

joblib.dump(X_scaler, 'data/scalers/X_scaler.save') 
joblib.dump(y_scaler, 'data/scalers/y_scaler.save') 

['data/scalers/y_scaler.save']

In [16]:
# Assuming your model takes input of the same shape as X_scaled
# Fetch a batch of data from the DataLoader
batch = next(iter(train_loader))

# The first part of the batch is your inputs (features)
dummy_input = batch[0]

torch.onnx.export(model, dummy_input, "data/models/simple_model.onnx")


================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

